In [ ]:
from seasalt.noise_to_salt.noise_to_salt import (
    NoiseDetector,
    weighted_mean_conv,
    calculate_wa_kernel,
    train_model,
    noise_adder,
    data_folder,
)
from functools import partial
from torch.nn import functional as F
import torch
from torchvision import datasets, transforms
from torch.utils.data import random_split
from torch.utils.data import DataLoader
import numpy as np

In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")
torch.manual_seed(101)
np.random.seed(101)
device

In [ ]:
transform = transforms.Compose(
    [
        transforms.Grayscale(),
        transforms.CenterCrop(321),
        transforms.ToTensor(),
    ]
)

wa_kerne = calculate_wa_kernel(27)


def collate_images(noise_parameter, noise_type, batch):
    images = [item[0] for item in batch]
    max_height = max(img.shape[1] for img in images)
    max_width = max(img.shape[2] for img in images)
    padded_images = [
        F.pad(img, (0, max_width - img.shape[2], 0, max_height - img.shape[1]))
        for img in images
    ]
    stacked_images = torch.stack(padded_images)
    noisy_images, masks = noise_adder(stacked_images, noise_parameter, noise_type)
    return noisy_images, masks


dataset = datasets.ImageFolder(root=str(data_folder), transform=transform)
lengths = [round(len(dataset) * 0.8), round(len(dataset) * 0.2)]
train_dataset, val_dataset = random_split(dataset, lengths)

train_dataloader = DataLoader(
    train_dataset,
    batch_size=8,
    shuffle=True,
    collate_fn=partial(collate_images, 0.01, "sap"),
)
val_dataloader = DataLoader(
    val_dataset,
    batch_size=8,
    shuffle=True,
    collate_fn=partial(collate_images, 0.01, "sap"),
)

In [ ]:
model = NoiseDetector()#.cuda()
model = model.to(device)

In [ ]:
train_model(model, 1e-3, train_dataloader, val_dataloader, device, "mask_unprocessed5", 100, True)